In [52]:
from torch.utils.cpp_extension import load
import statistics
bench_histogram = load(name="bench_histogram", sources=["cpp/bench_histogram_entry.cpp", "cpp/bench_histogram.cu"], extra_cuda_cflags=["-arch=sm_89", "--keep", "--keep-dir", "/workspace/benchmark/temp"], verbose=True)
print(bench_histogram)

import torch
blocks = torch.cuda.get_device_properties(0).multi_processor_count
outs = torch.zeros((1024*32), dtype=torch.int32, device="cuda")

for bins in (1, 2, 4, 8, 16, 32, 64, 128, 256):
    for items_per_thread in (128, 256, 1024, 65536,):
        for n_threads in (128,256):
            clocks = []
            for i in range(100):
                outs.zero_()
                result = bench_histogram.bench_histogram(outs, bins, items_per_thread, n_threads)
                if result == 1:
                    print(f"No kernel for ")
                    break

                outs_list = [int(a) for a in outs[:(n_threads//32)*blocks].tolist()]
                if outs_list.count(-1) != 0:
                    print(f"FOUND {outs_list.count(-1)} -1 VALUES!!!")
                    print(outs_list)
                    raise AssertionError
                elif outs_list.count(0) != 0:
                    print(f"FOUND {outs_list.count(0)} 0 VALUES")
                    print(outs_list)
                    raise AssertionError

                clocks.extend(outs_list)
            if not clocks: continue
            mean = int(statistics.mean(clocks))
            print(f"FOR {bins=}\t{items_per_thread=}\t{n_threads=}\tcycles/item: {int(mean/items_per_thread/32)}")
            # print(f"{outs_list[:100]}")
            # results[n_threads][f"{op_name}_{dtype}_{strat}"].append(mean)
            if mean < 1000:
                pass # print("\n\nTHIS OP AND DTYPE ARE PROBABLY EMPTY\n\n")

Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
The input conditions for extension module bench_histogram have changed. Bumping to version 26 and re-building as bench_histogram_v26...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu118/bench_histogram/build.ninja...
Building extension module bench_histogram_v26...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /usr/local/cuda/bin/nvcc  -DTORCH_EXTENSION_NAME=bench_histogram_v26 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.10/dist-packages/torch/include -isystem /usr/local/lib/python3.10/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.10/dist-packages/torch/include/TH -isystem /usr/local/lib/python3.10/dist-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /usr/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_89,code=compute_89 -gencode=arch=compute_89,code=sm_89 --compiler-options '-fPIC' -arch=sm_89 --keep --keep-dir /workspace/benchmark/temp -std=c++17 -c /workspace/benchmark/cpp/bench_histogram.cu -o bench_histogram.cuda.o 
FAILED: bench_histo

RuntimeError: Error building extension 'bench_histogram_v26'